In [132]:
import pandas as pd
import numpy as np
import random

from typing import List, Tuple
from numpy import shape
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [33]:
df = pd.read_csv('../datasets/cleaned_RNN_dataset.zip')
df.head()

,Unnamed: 0,track_id,track_name,track_artist,lyrics,track_album_id,track_album_name,playlist_name,playlist_id,playlist_genre,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,trees singing wind sky blue angels smiled saw ...,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,hard rock workout,3YouF0u7waJnolytf9JCXf,rock,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,na yeah spyderman freeze full effect uh huh re...,6oZ6brjB8x3GoeSYdwJdPc,Gold,back day r b new jack swing swingbeat rnb etc,3a9y4eeCJRmG9p4YKfqYIx,r&b,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,really can t stay baby cold outside i ve got g...,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,christmas soul,6FZYc2BvF7tColxO8PBShV,r&b,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,get business keep turning witness criminal fee...,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,k party dance mix,37i9dQZF1DX4RDXswvP6Mj,pop,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,hold breath look down keep trying darling okay...,3GNzXsFbzdwM0WKCZtgeNP,Chapters,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,...,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [34]:
df.shape

(15405, 24)

In [35]:
rap_df = df[df['playlist_name'] == 'southern hip hop']
cols = ['lyrics', 'track_album_id', 'track_album_id', 'language']
rap_df = rap_df.drop(cols, axis=1)
rap_df.head()

,Unnamed: 0,track_id,track_name,track_artist,track_album_name,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
55,72,02BDSCfu9jBYvJ8LkUVttD,Hi Life,UGK,Ridin' Dirty,southern hip hop,3R5c1OfJY7Rzl3p28pzGtL,rap,southern hip hop,0.663,...,5,-7.552,1,0.203,0.5790,0.0,0.217,0.613,79.721,325373
63,80,02GwPAncasVL82yC7y2hmN,Raise Up,Petey Pablo,Diary of a Sinner: 1st Entry,southern hip hop,4lcyWQDOzPfcbZrcBI3FOW,rap,southern hip hop,0.735,...,1,-5.215,1,0.167,0.1420,0.0,0.319,0.742,156.069,286427
89,110,03b1NUHnoavfpLLKcEMfIb,Go Getta,Jeezy,The Inspiration,southern hip hop,4lcyWQDOzPfcbZrcBI3FOW,rap,southern hip hop,0.397,...,3,-6.584,0,0.143,0.1050,0.0,0.150,0.431,174.039,229347
164,202,05vpswcCkiXX2egk5GBo2y,Middle Of The Night,MJG,No More Glory,southern hip hop,2NMW1nwQYSVlXd26uLenX6,rap,southern hip hop,0.856,...,6,-7.041,0,0.178,0.0917,0.0,0.306,0.781,137.997,305440
166,204,05xbEV5M8DAQmhFKKlHaDd,Bring It On,Geto Boys,Til Death Do Us Part,southern hip hop,2NMW1nwQYSVlXd26uLenX6,rap,southern hip hop,0.855,...,2,-9.000,1,0.331,0.1750,0.0,0.651,0.533,92.888,496133


In [36]:
rap_df.shape

(211, 21)

In [37]:
rap_df.describe()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000
mean,9235.213270,0.743720,0.691355,5.521327,-6.868915,0.554502,0.248878,0.110417,0.004481,0.228555,0.595882,120.858682,268436.014218
std,5352.437027,0.128614,0.138430,3.798906,2.445386,0.498203,0.108257,0.125760,0.050035,0.177515,0.193633,33.498699,56350.903962
min,72.000000,0.286000,0.330000,0.000000,-15.538000,0.000000,0.033500,0.000047,0.000000,0.027700,0.115000,72.437000,133933.000000
25%,4639.000000,0.678000,0.589000,1.500000,-8.077000,0.000000,0.174000,0.016150,0.000000,0.096600,0.464500,88.961000,235160.000000
50%,9257.000000,0.783000,0.697000,6.000000,-6.419000,1.000000,0.250000,0.062900,0.000000,0.162000,0.622000,122.208000,264507.000000
75%,13713.000000,0.834500,0.806000,9.000000,-5.300500,1.000000,0.321000,0.167500,0.000002,0.325500,0.728500,150.027000,301020.000000
max,18104.000000,0.974000,0.963000,11.000000,-0.804000,1.000000,0.605000,0.626000,0.718000,0.822000,0.977000,191.692000,496133.000000


audio_cols is a list of the audio metrics I want to work with.

In [38]:
audio_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']

In the below code: 
1. I'm grouping the data frame based on the playlist ID's giving me each playlist as its own group.
2. Looping through each playlist and formatting them to 36 songs. So each playlist has a consistent 36 songs in them.
    - Playlists that have more than 36 songs are randomly reduced to 36.
    - Playlist that have less than 36 songs are padded with the average of the original playlist values until they are 36 songs.

In [ ]:
# Group songs by playlist_id
grouped = df.groupby('playlist_id')

# Create an empty list to store the reshaped data
data = []

# 36 Was chosen because it was the average number of songs in each playlist.
max_songs_per_playlist = 36
n_features = 12

# Loop over each playlist and reshape the songs into rows
for playlist_id, group in grouped:
    # Get the songs for the current playlist and sort them by song_id
    songs = group.sort_values(by='track_id')
    
    # Calculate the number of songs in the playlist
    n_songs = songs.shape[0]

    # Calculate the number of rows we need for the playlist (assuming we want 50 songs per playlist)
    n_rows = np.ceil(n_songs / max_songs_per_playlist).astype(int)
    songs = pd.concat([songs]*n_rows).reset_index(drop=True)
    # Add the playlist_id to each row
    songs['playlist_id'] = playlist_id
       
    # Dropping all columns except for the audio metrics
    songs = songs.loc[:, audio_cols]
    
    # Determine the number of songs in current playlist
    num_songs_in_playlist = songs.shape[:2][0]
    
    if num_songs_in_playlist < max_songs_per_playlist:
        songs = songs.reindex(range(36), fill_value=0)
    elif num_songs_in_playlist > max_songs_per_playlist:
        songs = songs.drop(songs.sample(songs.shape[0] - 36).index)
    
    col_means = songs.mean()
    songs = songs.replace(0, col_means)
        
    # Add the playlist_id to each row
    songs['playlist_id'] = playlist_id
    data.append(songs)

In [124]:
def create_dependent_independent_vars(audio_metrics: np.ndarray, target_song_index: int):
    """
    Given a playlist, creates the dependent and independent variables (target data)

    Args:
    - audio_metrics: A 2D numpy array of shape (N, M), where N is the number of songs
        in the playlist and M is the number of audio metrics.
    - target_song_index: The index of the song to predict
    - target_song_audio_metrics: A 1D numpy array of shape (M, ) containing the audio metrics for the target song.

    Returns:
    - X: A 2D array containing the dependent variables, i.e the metrics of the audio data the model will use to determine the target variable.
    -Y: A 1D array containing the target auido metrics that the model is trying to predict.
    
    """
    # Remove the target song from the audio metrics array. The axis is set to zero to ensure that a row is removed and not a column.
    X = np.delete(audio_metrics, target_song_index, axis=0)

    # Remove the target song from the list of audio metrics
    Y = audio_metrics[target_song_index]

    return X, Y

The below code:
1. Loops through each playlist
2. Chooses a random song as the target variable
3. Delegates the rest of the songs as data the model will train on
4. Appends each dependent and independent variable match to a list

In [127]:
NUM_SONGS = 36

x_train = []
y_train = []

for i in range(len(data)):
    target_song_index = random.randint(0, NUM_SONGS-1)
    df = data[i]
    X, y = create_dependent_independent_vars(audio_metrics=df[audio_cols].values, target_song_index=target_song_index)
    x_train.append(X)
    y_train.append(y)

In [136]:
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape)
print(y_train.shape)

(423, 35, 12)
(423, 12)


In [139]:
model = Sequential()
model.add(LSTM(128, input_shape=(35,12)))
model.add(Dense(12, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd')

model.fit(x_train, y_train, epochs=3, batch_size=16)

Epoch 1/3
27/27 [==============================] - 2s 15ms/step - loss: -435942208.0000
Epoch 2/3
27/27 [==============================] - 0s 13ms/step - loss: -1361031040.0000
Epoch 3/3
27/27 [==============================] - 0s 12ms/step - loss: -2287344384.0000


In [ ]:
df.info()